## Team Member Names: Madeline Witters {-}

## Project Title: Predicting Customer Churn and Identifying Attributes of At-Risk Customers {-}

In [1]:
import pandas as pd

In [ ]:
data = pandas.read_csv("Bank Customer Churn Prediction", header=True)